In [1]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Basic Transformations'). \
    master('yarn'). \
    getOrCreate()

In [2]:
from pyspark.sql.functions import *

In [3]:
airtraffic_path = "/public/airtraffic_all/airtraffic-part/flightmonth=200801"

In [4]:
airtraffic = spark. \
    read. \
    parquet(airtraffic_path)

##### Get number of flights in the month of 2008 January.

In [5]:
airtraffic.count()

605659

In [6]:
airtraffic.select(count("*").alias('count')).show()

+------+
| count|
+------+
|605659|
+------+



In [7]:
airtraffic.select(count(lit(1)).alias('count')).show()

+------+
| count|
+------+
|605659|
+------+



In [8]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    describe(). \
    show()

+-------+------+------+------------------+
|summary|  Year| Month|        DayOfMonth|
+-------+------+------+------------------+
|  count|605659|605659|            605659|
|   mean|2008.0|   1.0|15.908469947610785|
| stddev|   0.0|   0.0| 8.994294747375292|
|    min|  2008|     1|                 1|
|    max|  2008|     1|                31|
+-------+------+------+------------------+



In [9]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    summary(). \
    show()

+-------+------+------+------------------+
|summary|  Year| Month|        DayOfMonth|
+-------+------+------+------------------+
|  count|605659|605659|            605659|
|   mean|2008.0|   1.0|15.908469947610785|
| stddev|   0.0|   0.0| 8.994294747375292|
|    min|  2008|     1|                 1|
|    25%|  2008|     1|                 8|
|    50%|  2008|     1|                16|
|    75%|  2008|     1|                24|
|    max|  2008|     1|                31|
+-------+------+------+------------------+



##### Get number of distinct dates from airtraffic data frame which is created using 2008 January data.

In [10]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth'). \
    distinct(). \
    count()

31

In [11]:
airtraffic. \
    select(countDistinct('Year', 'Month', 'DayOfMonth').alias('distinct dates')). \
    show()

+--------------+
|distinct dates|
+--------------+
|            31|
+--------------+



##### Get the total bonus amount from employees data set. 

In [12]:
employees = [(1, "Scott", "Tiger", 1000.0, 10,
                      "united states", "+1 123 456 7890", "123 45 6789"
                     ),
                     (2, "Henry", "Ford", 1250.0, None,
                      "India", "+91 234 567 8901", "456 78 9123"
                     ),
                     (3, "Nick", "Junior", 750.0, '',
                      "united KINGDOM", "+44 111 111 1111", "222 33 4444"
                     ),
                     (4, "Bill", "Gomes", 1500.0, 10,
                      "AUSTRALIA", "+61 987 654 3210", "789 12 6118"
                     )
                ]

In [13]:
employeesDF = spark. \
    createDataFrame(employees,
                    schema="""employee_id INT, first_name STRING, 
                    last_name STRING, salary FLOAT, bonus STRING, nationality STRING,
                    phone_number STRING, ssn STRING"""
                   )

In [14]:
employeesDF.show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [15]:
employeesDF. \
    select(sum(
                coalesce(col("bonus").cast('int'), lit(0)) * col("salary") / 100
              ).alias('total_bonus')
          ). \
    show()

+-----------+
|total_bonus|
+-----------+
|      250.0|
+-----------+



In [16]:
employeesDF. \
    selectExpr('sum((coalesce(cast(bonus AS INT), 0) * salary) / 100) AS total_bonus'). \
    show()

+-----------+
|total_bonus|
+-----------+
|      250.0|
+-----------+



##### Get revenue generated for a given order from order_items.

In [17]:
order_items = spark.read.json('/public/retail_db_json/order_items')

In [18]:
order_items.show(10, truncate=False)

+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_product_price|order_item_quantity|order_item_subtotal|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|1            |1                  |957                  |299.98                  |1                  |299.98             |
|2            |2                  |1073                 |199.99                  |1                  |199.99             |
|3            |2                  |502                  |50.0                    |5                  |250.0              |
|4            |2                  |403                  |129.99                  |1                  |129.99             |
|5            |4                  |897                  |24.99                   |2                  |49.98              |
|6            |4

In [25]:
order_id = input('Enter order_id:')

Enter order_id: 2


In [26]:
order_items. \
    filter(f'order_item_order_id = {int(order_id)}'). \
    show()

+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_product_price|order_item_quantity|order_item_subtotal|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|            2|                  2|                 1073|                  199.99|                  1|             199.99|
|            3|                  2|                  502|                    50.0|                  5|              250.0|
|            4|                  2|                  403|                  129.99|                  1|             129.99|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+



In [27]:
order_items. \
    filter(f'order_item_order_id = {int(order_id)}'). \
    select(sum("order_item_subtotal").alias("order_revenue")). \
    show()

+-------------+
|order_revenue|
+-------------+
|       579.98|
+-------------+



In [28]:
order_items. \
    filter(col('order_item_order_id') == lit(int(order_id))). \
    select(sum('order_item_subtotal').alias('order_revenue')). \
    show()

+-------------+
|order_revenue|
+-------------+
|       579.98|
+-------------+



#### Aggregate data using groupBy

##### Get number of flights scheduled each day for the month of January 2008

In [29]:
airtraffic. \
    groupBy(to_date(concat("year",
                           lpad("Month", 2, "0"),
                           lpad("DayOfMonth", 2, "0")
                          ) , 'yyyyMMdd'
                  ).alias("FlightDate")
           ). \
    count(). \
    show(31)

+----------+-----+
|FlightDate|count|
+----------+-----+
|2008-01-20|18653|
|2008-01-25|20313|
|2008-01-26|16276|
|2008-01-17|20273|
|2008-01-01|19175|
|2008-01-09|19820|
|2008-01-02|20953|
|2008-01-14|20176|
|2008-01-30|19766|
|2008-01-24|20257|
|2008-01-06|19893|
|2008-01-12|16572|
|2008-01-08|19603|
|2008-01-19|16249|
|2008-01-29|19485|
|2008-01-22|19504|
|2008-01-03|20937|
|2008-01-21|20133|
|2008-01-04|20929|
|2008-01-18|20347|
|2008-01-11|20349|
|2008-01-07|20341|
|2008-01-27|18903|
|2008-01-05|18066|
|2008-01-10|20297|
|2008-01-23|19769|
|2008-01-13|18946|
|2008-01-16|19764|
|2008-01-31|20260|
|2008-01-15|19503|
|2008-01-28|20147|
+----------+-----+



In [30]:
airtraffic. \
    groupBy(to_date(concat("year",
                           lpad("Month", 2, "0"),
                           lpad("DayOfMonth", 2, "0")
                          ) , 'yyyyMMdd'
                  ).alias("FlightDate")
           ). \
    agg(count(lit(1)).alias("FlightCount")). \
    show(31)

+----------+-----------+
|FlightDate|FlightCount|
+----------+-----------+
|2008-01-20|      18653|
|2008-01-25|      20313|
|2008-01-26|      16276|
|2008-01-17|      20273|
|2008-01-01|      19175|
|2008-01-09|      19820|
|2008-01-02|      20953|
|2008-01-14|      20176|
|2008-01-30|      19766|
|2008-01-24|      20257|
|2008-01-06|      19893|
|2008-01-12|      16572|
|2008-01-08|      19603|
|2008-01-19|      16249|
|2008-01-29|      19485|
|2008-01-22|      19504|
|2008-01-03|      20937|
|2008-01-21|      20133|
|2008-01-04|      20929|
|2008-01-18|      20347|
|2008-01-11|      20349|
|2008-01-07|      20341|
|2008-01-27|      18903|
|2008-01-05|      18066|
|2008-01-10|      20297|
|2008-01-23|      19769|
|2008-01-13|      18946|
|2008-01-16|      19764|
|2008-01-31|      20260|
|2008-01-15|      19503|
|2008-01-28|      20147|
+----------+-----------+



##### Get count of flights departed, total departure delay and average departure delay for each day over the month of January 2008

In [31]:
airtraffic. \
    filter('Cancelled = 0'). \
    groupBy(to_date(concat("year",
                           lpad("Month", 2, "0"),
                           lpad("DayOfMonth", 2, "0")
                          ) , 'yyyyMMdd'
                  ).alias("FlightDate")
           ). \
    agg(
        count(lit(1)).alias("FlightCount"),
        sum('DepDelay').alias('TotalDepDelay'),
        round(avg('DepDelay'), 2).alias('AverageDepDelay')
    ). \
    show(31)

+----------+-----------+-------------+---------------+
|FlightDate|FlightCount|TotalDepDelay|AverageDepDelay|
+----------+-----------+-------------+---------------+
|2008-01-20|      18406|     117460.0|           6.38|
|2008-01-25|      19787|     229850.0|          11.62|
|2008-01-26|      15860|      92129.0|           5.81|
|2008-01-17|      19401|     341271.0|          17.59|
|2008-01-01|      18623|     354108.0|          19.01|
|2008-01-09|      19443|      89595.0|           4.61|
|2008-01-02|      20442|     452979.0|          22.16|
|2008-01-14|      19267|      98261.0|            5.1|
|2008-01-30|      19072|     129345.0|           6.78|
|2008-01-24|      19935|     158134.0|           7.93|
|2008-01-06|      19210|     323214.0|          16.83|
|2008-01-12|      16346|      24876.0|           1.52|
|2008-01-08|      19140|     200670.0|          10.48|
|2008-01-19|      15373|     155488.0|          10.11|
|2008-01-29|      18596|     184855.0|           9.94|
|2008-01-2

##### Using order_items, get revenue for each order.

In [32]:
order_items. \
    groupBy('order_item_order_id'). \
    sum('order_item_subtotal'). \
    show()

+-------------------+------------------------+
|order_item_order_id|sum(order_item_subtotal)|
+-------------------+------------------------+
|                 29|                 1109.85|
|                474|       774.8199999999999|
|                964|       739.8800000000001|
|               1677|       649.9200000000001|
|               1806|                  789.94|
|               1950|      1015.8700000000001|
|               2214|                  449.96|
|               2250|                  889.94|
|               2453|       999.9300000000001|
|               2509|                  889.94|
|               2529|                   59.99|
|               2927|       999.9100000000001|
|               3091|      469.93000000000006|
|               3764|                   95.98|
|               4590|                  949.83|
|               4894|                  899.94|
|               5385|                  629.86|
|               5409|       699.9200000000001|
|            

In [33]:
order_items. \
    groupBy('order_item_order_id'). \
    agg(round(sum('order_item_subtotal'), 2).alias('revenue_per_order')). \
    show()

+-------------------+-----------------+
|order_item_order_id|revenue_per_order|
+-------------------+-----------------+
|                 29|          1109.85|
|                474|           774.82|
|                964|           739.88|
|               1677|           649.92|
|               1806|           789.94|
|               1950|          1015.87|
|               2214|           449.96|
|               2250|           889.94|
|               2453|           999.93|
|               2509|           889.94|
|               2529|            59.99|
|               2927|           999.91|
|               3091|           469.93|
|               3764|            95.98|
|               4590|           949.83|
|               4894|           899.94|
|               5385|           629.86|
|               5409|           699.92|
|               6721|           139.99|
|               7225|           774.86|
+-------------------+-----------------+
only showing top 20 rows



##### Get min and max order_item_subtotal for each order id.

In [34]:
order_items. \
    groupBy('order_item_order_id'). \
    agg(
        round(sum('order_item_subtotal'), 2).alias('revenue_per_order'),
        min('order_item_subtotal').alias('order_item_subtotal_min'),
        max('order_item_subtotal').alias('order_item_subtotal_max')
    ). \
    show(10)

+-------------------+-----------------+-----------------------+-----------------------+
|order_item_order_id|revenue_per_order|order_item_subtotal_min|order_item_subtotal_max|
+-------------------+-----------------+-----------------------+-----------------------+
|                 29|          1109.85|                 129.99|                 399.98|
|                474|           774.82|                  24.99|                 199.99|
|                964|           739.88|                 129.99|                 299.98|
|               1677|           649.92|                   50.0|                  250.0|
|               1806|           789.94|                  150.0|                 399.98|
|               1950|          1015.87|                  87.96|                 399.98|
|               2214|           449.96|                   50.0|                 399.96|
|               2250|           889.94|                  59.99|                 399.98|
|               2453|           

#### RollUp

#### Sorting Data Frames

##### Get daily count of cancelled flights where data is sorted in ascending order by count of cancelled flights.

In [35]:
airtraffic. \
    filter('cancelled = 1'). \
    groupBy(to_date(concat("year",
                           lpad("Month", 2, "0"),
                           lpad("DayOfMonth", 2, "0")
                          ) , 'yyyyMMdd'
                  ).alias("FlightDate")
           ). \
    agg(count(lit(1)).alias('CancelledFlightCount')). \
    orderBy('CancelledFlightCount'). \
    show(10)

+----------+--------------------+
|FlightDate|CancelledFlightCount|
+----------+--------------------+
|2008-01-12|                 226|
|2008-01-18|                 230|
|2008-01-20|                 247|
|2008-01-15|                 299|
|2008-01-24|                 322|
|2008-01-10|                 341|
|2008-01-13|                 359|
|2008-01-09|                 377|
|2008-01-26|                 416|
|2008-01-05|                 456|
+----------+--------------------+
only showing top 10 rows



In [36]:
airtraffic. \
    filter('cancelled = 1'). \
    groupBy(to_date(concat("year",
                           lpad("Month", 2, "0"),
                           lpad("DayOfMonth", 2, "0")
                          ) , 'yyyyMMdd'
                  ).alias("FlightDate")
           ). \
    agg(count(lit(1)).alias('FlightCount')). \
    orderBy(col('FlightCount').asc()). \
    show(10)

+----------+-----------+
|FlightDate|FlightCount|
+----------+-----------+
|2008-01-12|        226|
|2008-01-18|        230|
|2008-01-20|        247|
|2008-01-15|        299|
|2008-01-24|        322|
|2008-01-10|        341|
|2008-01-13|        359|
|2008-01-09|        377|
|2008-01-26|        416|
|2008-01-05|        456|
+----------+-----------+
only showing top 10 rows



##### Get daily count of cancelled flights where data is sorted in descending order by count of cancelled flights.

In [37]:
airtraffic. \
    filter('cancelled = 1'). \
    groupBy(to_date(concat("year",
                           lpad("Month", 2, "0"),
                           lpad("DayOfMonth", 2, "0")
                          ) , 'yyyyMMdd'
                  ).alias("FlightDate")
           ). \
    agg(count(lit(1)).alias('FlightCount')). \
    orderBy(col('FlightCount').desc()). \
    show(10)

+----------+-----------+
|FlightDate|FlightCount|
+----------+-----------+
|2008-01-31|       1081|
|2008-01-14|        909|
|2008-01-29|        889|
|2008-01-19|        876|
|2008-01-17|        872|
|2008-01-22|        788|
|2008-01-04|        769|
|2008-01-30|        694|
|2008-01-06|        683|
|2008-01-28|        654|
+----------+-----------+
only showing top 10 rows



##### Project Year, Month, DayOfMonth, CRSDepTime and Origin for this task.

##### Sort the data based up on year, month, day of month and then using scheduled time. Data should be sorted in ascending order by year, month and day of month then in descending order by scheduled time.

In [38]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth', 'CRSDepTime', 'Origin'). \
    orderBy('Year', 'Month', 'DayOfMonth', 'CRSDepTime'). \
    show(10)

+----+-----+----------+----------+------+
|Year|Month|DayOfMonth|CRSDepTime|Origin|
+----+-----+----------+----------+------+
|2008|    1|         1|        10|   LAX|
|2008|    1|         1|        15|   SMF|
|2008|    1|         1|        25|   SMF|
|2008|    1|         1|        25|   PHX|
|2008|    1|         1|        30|   LAX|
|2008|    1|         1|        30|   ONT|
|2008|    1|         1|        30|   LAS|
|2008|    1|         1|        30|   ANC|
|2008|    1|         1|        35|   MCO|
|2008|    1|         1|        35|   SFO|
+----+-----+----------+----------+------+
only showing top 10 rows



In [39]:
airtraffic. \
    select('Year', 'Month', 'DayOfMonth', 'CRSDepTime', 'Origin'). \
    orderBy('Year', 'Month', 'DayOfMonth', col('CRSDepTime').desc()). \
    show(10)

+----+-----+----------+----------+------+
|Year|Month|DayOfMonth|CRSDepTime|Origin|
+----+-----+----------+----------+------+
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   PHX|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   SLC|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   TUS|
|2008|    1|         1|      2359|   LAS|
|2008|    1|         1|      2359|   LAS|
+----+-----+----------+----------+------+
only showing top 10 rows



##### Get employees data in ascending order by nationality. However, data related to United States should come at top always.

In [40]:
employeesDF.show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [41]:
employeesDF. \
    withColumn('sort_column', when(upper(col('nationality')) == 'UNITED STATES', 0).otherwise(1)). \
    orderBy('sort_column', 'nationality'). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|sort_column|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|          0|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|          1|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|          1|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|          1|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+



In [42]:
employeesDF. \
    withColumn(
        'sort_column', 
        expr("""CASE WHEN upper(nationality) = 'UNITED STATES' THEN 0 else 1 END""")
    ). \
    orderBy('sort_column', 'nationality'). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|sort_column|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|          0|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|          1|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|          1|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|          1|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+-----------+



##### Sort the data in employeesDF using bonus. Data should be sorted numerically and null and empty values should come at the end.

In [43]:
employeesDF. \
    orderBy('bonus'). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [44]:
employeesDF. \
    orderBy(col('bonus').cast('int')). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+



In [45]:
employeesDF. \
    orderBy(employeesDF.bonus.cast('int').asc_nulls_last()). \
    show()

+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|employee_id|first_name|last_name|salary|bonus|   nationality|    phone_number|        ssn|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+
|          4|      Bill|    Gomes|1500.0|   10|     AUSTRALIA|+61 987 654 3210|789 12 6118|
|          1|     Scott|    Tiger|1000.0|   10| united states| +1 123 456 7890|123 45 6789|
|          3|      Nick|   Junior| 750.0|     |united KINGDOM|+44 111 111 1111|222 33 4444|
|          2|     Henry|     Ford|1250.0| null|         India|+91 234 567 8901|456 78 9123|
+-----------+----------+---------+------+-----+--------------+----------------+-----------+

